In [1]:
! pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

import FinanceDataReader as fdr
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [6]:
start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (235, 1)


# Final

In [7]:
! pip install pmdarima

     |████████████████████████████████| 1.4 MB 26.7 MB/s 
     |████████████████████████████████| 9.8 MB 57.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [8]:
nmae_df = pd.DataFrame(columns = ['code', 'nmae'])

In [9]:
submission_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,submission_name))

In [10]:
from pmdarima.arima import auto_arima
import warnings
import time

warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  print(f'\n code: {code} \n')

  # Load data
  stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data.fillna(method = 'ffill', inplace = True)

  # Split the data into train / validation / test sets
  train_public = stock_data[:-20] # ~ 10/29 : predict from 11/1 to 11/5
  train_private = stock_data # ~ 11/26 : predict from 11/29 to 12/3

  test_public = stock_data[-20:-15]['Close'].reset_index(drop = True) # from 11/1 to 11/5
  test_public = np.array(test_public)

  # Fit the model with selected p, d, q values
  public_model = auto_arima(train_public['Close'].values.astype('float64'), 
                            start_p = 1, start_q = 1,
                            start_P = 0, start_Q = 0,
                            m = 73,
                            error_action = 'ignore', suppress_warnings = True)
  public_model_fitted = public_model.fit(train_public['Close'].values.astype('float64'))
  public_forecast_result = public_model_fitted.predict(n_periods = 5)

  pred_public = public_forecast_result

  # Check nmae values
  public_nmae = np.mean(np.abs(test_public - pred_public) / test_public)
  nmae_df = nmae_df.append({'code' : code, 'nmae' : public_nmae}, ignore_index = True)
    
  print('public predicted values: ', pred_public, '\n')
  print('nmae: ', public_nmae, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i, code] = pred_public[i]

  private_model = auto_arima(train_private['Close'].values.astype('float64'),
                             start_p = 1, start_q = 1,
                             start_P = 0, start_Q = 0,
                             m = 73,
                             error_action = 'ignore', suppress_warnings = True)
  private_model_fitted = private_model.fit(train_private['Close'].values.astype('float64'))
  private_forecast_result = private_model_fitted.predict(n_periods = 5)

  pred_private = private_forecast_result

  print('private predicted values: ', pred_private, '\n')

  # Add predicted valued to submission file
  for i in range (0, 5):
    submission.loc[i+5, code] = pred_private[i]

  time.sleep(1)

private predicted values:  [6683.49081069 6697.30534908 6747.42030834 6732.23231438 6741.64312656] 



 64%|██████▎   | 235/370 [7:40:37<9:18:10, 248.07s/it]


 code: 000250 

public predicted values:  [47050. 47050. 47050. 47050. 47050.] 

nmae:  0.058865031405902477 

private predicted values:  [44170.51007173 43970.37243169 43837.70201964 43640.60168144
 43505.03070361] 



 64%|██████▍   | 236/370 [7:41:39<7:09:14, 192.20s/it]


 code: 048410 

public predicted values:  [23500. 23500. 23500. 23500. 23500.] 

nmae:  0.04468727982499273 

private predicted values:  [20750. 20750. 20750. 20750. 20750.] 



...

 99%|█████████▉| 368/370 [10:16:49<01:43, 51.55s/it]


 code: 110790 

public predicted values:  [42850. 42850. 42850. 42850. 42850.] 

nmae:  0.024161457243282246 

private predicted values:  [47676.12052112 47787.35794454 47889.80174311 47993.15403148
 48096.41246166] 



100%|█████████▉| 369/370 [10:17:53<00:55, 55.17s/it]


 code: 099320 

public predicted values:  [50435.84528196 50514.9386696  50732.16066186 50802.21454228
 51002.32475011] 

nmae:  0.049029677426551385 

private predicted values:  [45538.37101062 45651.63213987 45803.46270977 45975.23557398
 46076.02945097] 



100%|██████████| 370/370 [10:22:53<00:00, 101.01s/it]


In [11]:
# nmae 평균
nmae_result = np.array(nmae_df['nmae'])

In [12]:
np.mean(nmae_result)

0.031156044224318814

In [13]:
submission.to_csv("/content/drive/MyDrive/stock_2/submission_arima.csv", index = False)